In [1]:
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
import cv2
from scipy.signal import find_peaks
from scipy.fftpack import ifft2
import pandas as pd
import time
# import websocket
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
# import mouse
from rotpy.system import SpinSystem
from rotpy.camera import CameraList
from rotpy.camera_nodes import CameraNodes
from numpy import unravel_index

In [2]:
def startDriver():
    """Starts the WebDriver (Chrome) and returns the driver object."""
    driver = webdriver.Chrome(executable_path=r"C:\Users\Webdriver\chromedriver.exe")
    return driver

def getToScreener(driver):
    """Navigates the WebDriver to the specified URL and returns the driver object."""
    URL = 'file:///C:/Users/Bruger/Documents/GitHub/B.Sc.-Touch-Free-Interaction/Website/index.html'
#     URL = 'file:///C:/Users/mkrhi/OneDrive/Dokumenter/GitHub/Touch-Free-Interaction/Website/index.html'
    driver.get(URL)
    return driver

def getToDarkness(driver):
    """Navigates the WebDriver to the specified URL and returns the driver object."""
#     URL = 'file:///C:/Users/mkrhi/OneDrive/Dokumenter/GitHub/Touch-Free-Interaction/Website/black.html'
    URL = 'file:///C:/Users/Bruger/Documents/GitHub/B.Sc.-Touch-Free-Interaction/Website/black.html'
    driver.get(URL)
    return driver

def clickNumber(number, driver):
    """Clicks the specified number on a web page using the WebDriver."""
    button = driver.find_element(By.ID, f'key{number}')
    button.click()

def on_message(ws, message):
    """Prints the received message through WebSocket."""
    print(message)

def power_spectrum(fft):
    """Returns the power spectrum of the 2D-Fourier Transform."""
    return np.abs(fft)**2

def FFT(image):
    """Returns the fast Fourier transform after it is centered."""
    fft = np.fft.fft2(image, s=None, axes=(-2, -1), norm=None)
    fft = np.fft.fftshift(fft, axes=None)
    return fft

def inverse_FFT(pwr_spectrum):
    """Computes the inverse Fourier transform of the power spectrum."""
    fft_array = np.fft.ifftshift(pwr_spectrum, axes=None)
    img = np.fft.ifft2(fft_array)
    #img = ifft2(fft_array)
    img = np.abs(img)
    print(np.max(img))
    img[img < 1] = 0
    img = (img - np.min(img)) * (255 / (np.max(img) - np.min(img)))
    return img.astype(np.uint8)

def maskDots(fft_img, X, Y, radius_squared):
    """Masks the dots in the FFT image."""
    r1 = (np.arange(crop_size)[:, None] - Y - crop_size/2) ** 2 + (np.arange(crop_size) - X - crop_size/2) ** 2
    r2 = (np.arange(crop_size)[:, None] + Y - crop_size/2) ** 2 + (np.arange(crop_size) + X - crop_size/2) ** 2
    mask = np.logical_and(r1 > radius_squared, r2 > radius_squared)
    fft_img[mask] = 0
    fft_copy = fft_img.copy()
    return fft_copy

def maskLine(fft_img, X, Y, radius_squared):
    """Masks the line in the FFT image."""
    r = []
    for i in range(10):
        r.append((np.arange(crop_size)[:, None] - Y + (2*Y/10*i) - crop_size/2) ** 2 + (np.arange(crop_size) - X + (2*X/10*i) - crop_size/2) ** 2)
    center = (np.arange(crop_size)[:, None] - crop_size/2) ** 2 + (np.arange(crop_size) - crop_size/2) ** 2
    mask = r[0] > radius_squared
    for i in range(9):
        if i not in [3, 4, 5, 6]:
            mask = np.logical_and(mask, r[i+1] > radius_squared)
    fft_img[mask] = 0
    fft_copy = fft_img.copy()
    return fft_copy

def maskSquare(a, radius, X, Y):
    """Masks the square in the FFT image."""
    X = int(X + crop_size/2)
    Y = int(Y + crop_size/2)
    b = np.array(a.copy(), dtype=np.uint8)
    return b[Y - radius:Y + radius, X - radius:X + radius]

def mask(fft_img, height, width, X, Y, radius_squared):
    """Masks the FFT image."""
    return maskDots(fft_img, height, width, X, Y, radius_squared)

def planeInteraction(peak):
    """Checks if the plane interaction meets a condition."""
    value = findPeakGrid(peak)
    # Paper
#     return value > 5 * 10 ** 6, value
    # Pointing-pen
#     return value > 3 * 10 ** 6, value
    # Finger
    return value > 1 * 10 ** 6, value

def keyAreaPressed(img):
    """Returns the pressed key based on the image."""
    keys, points = 3, 10
    height, width = len(img) / keys, len(img[0]) / keys
    total = []
    for h in range(keys-1, -1, -1):
        for w in range(keys):
            value = 0
            for i in range(points):
                for j in range(points):
                    value += img[int(h*height + height/4 + height/2*(i+1)/points)][int(w*width + width/4 + width/2*(i+1)/points)]
            total.append(value)
    multiply_factor = [0.9, 1, 1.6, 1.5, 1.4, 1.4, 1.6, 1.1, 1.2] # Amplifying each key intensity accorting to testing
    for i in range(len(total)):
        total[i] = total[i] * multiply_factor[i]
    ID = f'key{total.index(max(total))+1}'
    return ID

def lowToHigh(currentB, previousB):
    """Checks if a condition is met by comparing two values."""
    passed = False
    if currentB and not previousB:
        passed = True
    return passed

k1 = -0.5  # pincushion distortion
k2 = 0.0
k3 = 0.0
p1 = 0.0
p2 = 0.0
dist_coeffs = np.array([k1, k2, k3, p1, p2], dtype=np.float32)

def distortImage(img):
    """Distorts the image."""
    focal_length = 300
    center_x = img.shape[1] / 2
    center_y = img.shape[0] / 2
    camera_matrix = np.array([[focal_length, 0, center_x],
                              [0, focal_length, center_y],
                              [0, 0, 1]], dtype=np.float32)
    img_distorted = cv2.undistort(img, camera_matrix, dist_coeffs)
    return img_distorted

def findPeakGrid(a):
    """Finds the peak value in a grid."""
    size = len(a)
    b = np.asarray(a)
    return b.max()

def xyPeak(a):
    """Returns the X and Y coordinates of the peak in a grid."""
    peak = findPeakGrid(a)
    return [peak[0]+X-radius, peak[1]+Y-radius]

def cameraInDarkness(cap):
    """Sets camera settings for darkness conditions."""
    cap.set(cv2.CAP_PROP_EXPOSURE, -2)
    cap.set(cv2.CAP_PROP_BRIGHTNESS, 104)
    cap.set(cv2.CAP_PROP_SATURATION, 0)
    cap.set(cv2.CAP_PROP_CONTRAST, 255)
    cap.set(cv2.CAP_PROP_SHARPNESS, 165)
    cap.set(cv2.CAP_PROP_FOCUS, 18)
    cap.set(cv2.CAP_PROP_GAIN, 255)
    cap.set(cv2.CAP_PROP_TILT, 0)
    cap.set(cv2.CAP_PROP_PAN, 0)
    cap.set(cv2.CAP_PROP_ZOOM, 390)

def cameraInLight(cap):
    """Sets camera settings for light conditions."""
    cap.set(cv2.CAP_PROP_EXPOSURE, -2)
    cap.set(cv2.CAP_PROP_BRIGHTNESS, 230)
    cap.set(cv2.CAP_PROP_CONTRAST, 255)
    cap.set(cv2.CAP_PROP_SHARPNESS, 130)
    cap.set(cv2.CAP_PROP_FOCUS, 18)
    cap.set(cv2.CAP_PROP_GAIN, 60)
    cap.set(cv2.CAP_PROP_TILT, -1)
    cap.set(cv2.CAP_PROP_PAN, 0)
    cap.set(cv2.CAP_PROP_ZOOM, 500)
    return cap

def getComputerCamera():
    """Returns the computer's camera object."""
    return cv2.VideoCapture(0)

def getWebCam():
    """Returns the webcam object."""
    return cv2.VideoCapture(1 + cv2.CAP_DSHOW)

def changeCameraSetting(cap):
    """Changes the camera settings."""
    cap.set(cv2.CAP_PROP_SETTINGS, 1)
    return cap

def captureImage(camera):
    """Captures an image from the camera."""
    camera.begin_acquisition()
    image_cam = camera.get_next_image(timeout=5)
    image = image_cam.deep_copy_image(image_cam)
    image_data = image_cam.get_image_data()
    image_height = image.get_height()
    image_width = image.get_width()
    image_cam.release()
    camera.end_acquisition()
    numpy_array = np.array(image_data)
    reshaped_array = numpy_array.reshape((image_height, image_width))
    reshaped_array = reshaped_array
    return resizeImg(reshaped_array)

def initCamera():
    """Initializes the camera."""
    system = SpinSystem()
    cameras = CameraList.create_from_system(system, update_cams=True, update_interfaces=True)
    camera = cameras.create_camera_by_index(0)
    camera.init_cam()
#     camera.camera_nodes.ExposureAuto.is_available()
#     print(camera.camera_nodes.Gain.get_entries_names())
#     camera.camera_nodes.GainAuto.set_node_value_from_str('Off')
#     camera.camera_nodes.AcquisitionFrameRateEnable.set_node_value_from_str('true')
#     camera.camera_nodes.AcquisitionFrameRate.set_node_value(50)
#     camera.camera_nodes.Gain.set_node_value(23.9)
#     camera.camera_nodes.GainAuto.set_node_value_from_str('Continuous')
#     camera.camera_nodes.ExposureAuto.get_entries_names()
    return camera

def resizeImg(img):
    """Resizes the image."""
    smallest_axis = min(len(img), len(img[0]))
    img = img[100:smallest_axis-100, 100:smallest_axis-100]
    dim = (crop_size, crop_size)
    resized = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
    return resized


In [3]:
camera = initCamera()

In [4]:
# Mask values to be edited
Y, X, radius = 7, 13, 6
radius_squared = int(radius ** 2)
crop_size = 300

In [5]:
# Start Camera
# cap = getWebCam()
# cameraInDarkness(cap)
# changeCameraSetting(cap)

In [6]:
## Open the keyboard using selenium
driver = startDriver()
keyboard = getToDarkness(driver)

previousClicked = False
firstRun = True
firstImageCaptured = False

while True:
    # Capture frame-by-frame
#     ret, frame = cap.read()
#     frame = frame[0:crop_size, 0:crop_size]

#     # Convert the frame to grayscale
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    while firstRun:
        camera = initCamera()
        first_gray = captureImage(camera)
        firstRun = False
        
    gray = captureImage(camera)
    
    # Capture background frame when user presses 'f'
    if cv2.waitKey(1) & 0xFF == ord('f'):
        first_gray = gray
        keyboard = getToScreener(driver)
        firstImageCaptured = True

    # Difference in images
    diff_img = cv2.absdiff(gray, first_gray)
    dist_img = distortImage(gray)
#     dist_img = diff_img

    
    fft_dist = FFT(dist_img)
    #only_mask = maskSquare(fft_dist, radius, X, Y)
    #peak = xyPeak(only_mask)
    
    masked_img = maskDots(fft_dist, X, Y, radius_squared)
    final_img = inverse_FFT(masked_img)
    
    #img1 = np.concatenate((gray,power_spectrum(FFT(gray))),axis=0)
    #img2 = np.concatenate((dist_img,power_spectrum(FFT(dist_img))),axis=0)
    #img3 = np.concatenate((final_img,power_spectrum(masked_img)),axis=0)
    #img_total = np.concatenate((img1,img2,img3),axis=1)
    #cv2.imshow('allWindows', img_total)
    
    cv2.imshow("gray",gray)
    cv2.imshow("power_spectrum",power_spectrum(FFT(gray)))
    cv2.imshow("difference",dist_img)
    cv2.imshow("dist_power",power_spectrum(fft_dist))
    cv2.imshow("final_img",final_img)
    ps_mask = power_spectrum(masked_img)
    cv2.imshow("masked_power",ps_mask)
    
    
    # Tell if plane is interacted with and which key is pressed
    if firstImageCaptured:
        currentClicked = planeInteraction(ps_mask)[0]
        print(planeInteraction(ps_mask)[1], end="\r")
        if currentClicked:
            #print((keyAreaPressed(final_img)), end="\r")
            if lowToHigh(currentClicked, previousClicked):
                keyboard.find_element(By.ID, keyAreaPressed(final_img)).click()
                #print(currentClicked, previousClicked)
                print(keyAreaPressed(final_img), end="\r")
        previousClicked = currentClicked

    # Exit if the user presses 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the VideoCapture object and close all windows
camera.deinit_cam()
camera.release()
cv2.destroyAllWindows()
keyboard.close()

C:\Users\Bruger\AppData\Local\Temp\ipykernel_18660\3596234844.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"C:\Users\Webdriver\chromedriver.exe")


2.472990929836328
2.358144651299357
2.328370510310822
2.268215120180706
2.3255684278801114
2.3726953982322896
2.4201863006402298
2.3537928812566
2.434499229293648
2.3800595886441935
2.3203261716131887
1.4112140736024077
1.4110596019272015
1.4004568736157756
1.4403860553773566
1.38090413476266
1.371883504231522
1.4036172692208873
0.14608997786790065
0.10764878029386286


C:\Users\Bruger\AppData\Local\Temp\ipykernel_18660\3596234844.py:47: RuntimeWarning: divide by zero encountered in double_scalars
  img = (img - np.min(img)) * (255 / (np.max(img) - np.min(img)))
C:\Users\Bruger\AppData\Local\Temp\ipykernel_18660\3596234844.py:47: RuntimeWarning: invalid value encountered in multiply
  img = (img - np.min(img)) * (255 / (np.max(img) - np.min(img)))


0.09755564344922764
0.10868394371249239
0.11769173007151704
0.13692187142909748
0.09151018290534835
0.09778988450470819
0.12459614314110641
0.12508211829985766
0.09109555550486145
0.14798272625953063
0.09821588989166061
0.13934780085874807
0.10598040354136905
0.14602536253765472
0.10345939597225091
0.11726768389609098
0.12219003317806017
0.10619420545308499
0.11152506168150486
0.11882298730602155
0.10408925293712526
0.11570268591221654
0.13274928761956006
0.11365354028432274
0.1069099810673569
0.1504730581331673
0.11489026077603134
0.11740693661443603
0.12047236104249306
2.7016515636857737
2.7131325573032465
2.7157391736962726
2.7549367668474445
2.7933501828602423
2.6952560843814384
2.737719138568536
2.738742816271863
2.686422972934882
2.7091339912480925
2.721537913319375
2.6771398618654207
2.717921230665671
2.7429136666419005
2.7573632347858794
2.707519378179853
2.7177915549987888
2.7479682466350783
2.7479850948407427
2.7045496372674487
2.7304466559658516
2.7455491464836457
2.75893286

1.6114974992660558
1.6129266136815927
1.6180202949959321
1.6400398946791652
1.6579494431972072
1.6653140760763734
1.5941020120711307
1.6163967779993665
1.6002268571025158
1.5870802248542848
1.6333157560929592
1.6197184967911062
1.6259523991735327
1.5558195970760218
1.6477132979329627
1.6085017966725667
1.6416189667226817
1.6123926764624223
1.5551450167202887
1.4162916674655714
1.6383894544334228
2.0243047885352197
2.0482741781605114
2.0514601888136395
2.0145047447989244
2.0333202814591975
1.4735112697888717
1.4456912098323458
1.4320784549772416
0.19655178967030423
0.21115780005269072
1.3364822305817743
0.17782901627880757
0.19459377286206528
0.5220435235893587
1.2724931221436138
1.3617703656049593
1.4033235199499916
1.3568555198548307
1.3512129115501852
1.2260189801868622
1.2992414858522612
1.2873734185793464
1.252044588597628
1.246595726464192
1.4457377046881896
0.8999968438693342
0.8790684912583842
0.8398114388083352
0.7755401878098254
0.9067843558207265
0.2110868086757196
0.47649417